<a href="https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/WOS_SCI_SCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WOS+SCI+SCP+PTJ+GS+LNS
Merge the bibliographic datasets for 
* Web of Science, 
* Scielo 
* Scopus 
* Google Scholar
* Puntaje
* Lens
of the scientific articles of Universidad de Antioquia

For details see [merge.ipynb in Colaboratory](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/merge.ipynb)

## functions

In [1]:
import unidecode
def get_author_info(x):
    sep='; '
    authors=[{'WOS_author':x[0].split(sep)[0],'affiliation':[x[0].split(sep)[-1]],'i':0}]
    iau=1
    for y  in x:
        y2=y.replace('[','').replace('] ',sep).split(sep)
        for z in y2[:-1]:
            aulist=[ d.get('WOS_author') for d in authors]
            if z not in aulist:
                authors.append({'WOS_author':z,'affiliation':[y2[-1]],'i':iau})
                iau=iau+1
            else:
                if y2[-1] not in [ d.get('affiliation') for d in authors if d.get('WOS_author')==z][0]:
                    index_author=[ d.get('i') for d in authors if d.get('WOS_author')==z][0]
                    authors[index_author]['affiliation'].append(y2[-1])
    return authors

def dictionary_list_add_columns(df,df_dl,df_dl_key,df_dl_i,df_columns):
    '''
    For a
     df: Pandas DataFrame 
    with a:
     df_dl: column of list of dictionaries, with
     df_dl_key: dictionary key: e.g x=[{df_dl_key:1},{df_dl_key:2}]
    for the element df_dl_i of the list:
    Update the dictionary with:
        df_dl_key==x[df_dl_i][df_dl_key]
    with the dictionaries { df_columns[i]: df_columns[i].values }
    '''
    dff=df.copy()
    for key in df_columns:
        tmp=dff[df_dl].combine(dff[key],
                func=lambda x,y: y if pd.isna(y) 
                                   else 
                                     [z.update({key:y}) 
                                     if z.get(df_dl_key)==x[df_dl_i][df_dl_key] 
                                     else z 
                                 for z in x  ] )
    return dff

def split_full_names(y,full_name='full_name'):
    """
    From an input dictionary with {full_name:'APPELLIDO1 APPELLIDO2 NOMBRES'}
    Obtain a dictionary with the several name parts.
    """    
    yy=y.get(full_name).title()
    lfn=len(y[full_name].split())
    aps=0
    d={ 'PRIMER APELLIDO':yy.split()[aps] }
    aps=aps+1
    if lfn>=4:
        names=-2
        if lfn==5: # Extra name or last name
            yyy=yy.split()
            ll=pd.np.array( [len(n) for n in yyy ] )
            if ll[3:][ ll[3:]  <= 3 ].shape[0]:
                # last_names first_first_name de(l) second_first_name
                yy=' '.join( [ y for y in yyy if len(y)>=3] )
            else: 
                # first_last name de(l) second_last_name first_names
                tmpll=yyy.pop() # internal memory
                yy=' '.join( yyy )  
        if len( d['PRIMER APELLIDO'] )<=3:
            d['PRIMER APELLIDO']=d['PRIMER APELLIDO']+' '+yy.split()[aps]
            aps=aps+1
            d.update(  {'SEGUNDO APELLIDO':yy.split()[aps]} )
            names=names+1
            
        d.update({'SEGUNDO APELLIDO':yy.split()[aps]})
        aps=aps+1
        if len( d['SEGUNDO APELLIDO'] )<=3:
            d['SEGUNDO APELLIDO']=d['SEGUNDO APELLIDO']+' '+yy.split()[aps]
            if names==-2:
                names=names+1
    elif lfn>=3:        
        d.update({'SEGUNDO APELLIDO':yy.split()[aps]})
        names=-1
    else: #Colombian interpretation (TODO: Includes Brazilian interpretation)    
        names=-1
    d.update({'NOMBRES':' '.join( yy.split()[names:]),
              'INICIALES':' '.join( [z[0]+'.' for z in yy.split()[names:]] ),
              })
    if not d.get('SEGUNDO APELLIDO'):
        d['SEGUNDO APELLIDO']=''
    #if not d.get('NOMBRE COMPLETO'):
    #    d['NOMBRE COMPLETO']=''        
    return d

# Creates mask Search key in a list of dictionay
# First apply convert null values to string
# Second apply: implement a mask
def find_key_in_list_of_dictionaries(df,column,key,pattern):
    return df[column].apply(lambda x: 
                [ '' if pd.isnull( y.get(key)) else y for y in x ]  ).apply(
                            lambda x: 
                [ True if y.get(key).find(pattern)>-1 else False for y in x  ][0]  )

def key_contains_in_list_of_dictionaries(df,pattern,column='authors_WOS',key='WOS_author'):
    #TODO: loop in column len
    i=0
    r=df[ df[column].str[i].apply(lambda x: {} if pd.isnull(x) else x).apply(
                       lambda x: x.get(key) if x else '').str.contains(
        pattern) ][column].reset_index(drop=True)
    return r

In [2]:
import wosplus as wp
import numpy as np
import pandas as pd
import unidecode
import os
from IPython.display import clear_output
pd.set_option('display.max_colwidth',200)

##  Configure public links of  files in Google Drive
* If it is a Google Spreadsheet the corresponding file is downloaded as CSV
* If it is in excel or text file the file is downloaded  directly

To define your  own labeled IDs for public google drive files edit the next cell:

In [111]:
%%writefile drive.cfg
[FILES]
UDEA_WOS.xlsx       = 1px2IcrjCrkyu7t78Q7PAE5nzV_yuPt9t
UDEA_SCI.xlsx       = 1pWMY5P72j0Ca6D-cm7dn7Q4TBGTs4PWV
UDEA_SCP.xlsx       = 1ulCsFHzDiTmuL9TH8F58ulh0u8Z2ylKh
UDEA_WOS_SCI_SCP.xlsx   = 1o9otmklgh-0w18Avv2ZTKOXr3vZbjwvj
UDEA_WOS_SCI_SCP.json=1RTDCh5pl0vapjJT_e9ZwadHPGBKGGv6Y
UDEA_WOS_SCI_SCP.json.gz=19E1C1kRk4I0V3uXojqko8-NEicWaPp1j
WOS_SCP_UDEA_SJR_SIU.xlsx=0BxoOXsn2EUNIQ3R4WDhvSzVLQ2s
Base_de_datos_investigadores_Definitiva.csv=12oalgUeKhpvzkTPBP8pXCeHTrF-KO223dy9ov9w9QKs
UDEA_authors_with_WOS_info.json=1o1eVT4JD0FMMICq_oxrTJOzWh47veBMw

Overwriting drive.cfg


##  Load data bases

In [4]:
affil='Univ Antioquia'
drive_files=wp.wosplus('drive.cfg')
RECOVER=False
if RECOVER==True:
    print('Go to: http://fisica.udea.edu.co:4443/user/restrepo/notebooks/prog/medicion/medicion/cienciometria/WOS_SCI_SCP_PTJ_GS_LNS.ipynb#Load-output-restuls-of-previous-Cell-runs')

#### DEBUG: if False stop in UDEA_PTJ!!!!

In [112]:
RECOVER_ALL=True
if not RECOVER_ALL:
    if os.path.exists('UDEAtmp.json'):
        UDEA=pd.read_json('UDEAtmp.json')
        UDEA=UDEA.reset_index(drop=True)    
else:
    if os.path.exists('UDEA_WOS_SCI_SCP.json'):
        UDEA=pd.read_json('UDEA_WOS_SCI_SCP.json')
    else:    
        UDEA=drive_files.read_drive_json('UDEA_WOS_SCI_SCP.json')

In [113]:
for t in UDEA.Tipo.unique():
    print( '{}:{}'.format( t, UDEA[ UDEA.Tipo==t].shape[0] ) )

SCI_SCP:1621
WOS_SCP:5823
WOS:1884
SCP:2584
WOS_SCI:147
SCI:2892
WOS_SCI_SCP:771


In [7]:
UDEA.shape

(15726, 151)

## Extract  affiliation from C1

In [8]:
UDEA_SCI=UDEA.SCI_C1.combine(UDEA.Tipo,func=lambda x,y: x if y.find('SCI')>-1 and y.find('WOS')==-1 else None)
UDEA['C1']=UDEA_SCI.combine(UDEA.C1,lambda x,y:x.replace(
    'Universidad de Antioquia',affil).replace(
    'Univ. de Antioquia',affil) if x else y)

In [9]:
i=0
import re
rename='[\w\-\.\s]+'
SCP2WOS=UDEA['SCP_Authors with affiliations'].str.replace(
                                    '; ','\n').str.replace(
                                    '^({},{}),'.format( rename,rename),r'[\1]',re.UNICODE).str.replace(
                                    '\n({},{}),'.format(rename,rename),r'\n[\1]',re.UNICODE).str.replace(
                                    '(,\s+\w\.)(\w\.\])'.format(rename,rename),r'\1 \2',re.UNICODE)
SCP2WOS.loc[i] 

''

In [10]:
SCP2WOS=SCP2WOS.str.replace('Universi[dadty]{2,3}\s+[deofDEOF]{2}\s+Antioqu[ií]a',affil
      ).str.replace('U[\.niv]{1,4}\s+[deofDEOF]{0,2}\s*Antioqu[ií]a',affil
      ).str.replace('Antioqu[ií]a\s+[deofDEOF]{0,2}\s*Universi[dadty]{2,3}',affil
      )

In [11]:
UDEA_SCP=SCP2WOS.combine(UDEA.Tipo,func=lambda x,y: x if y=='SCP' else None)
UDEA['C1']=UDEA_SCP.combine(UDEA.C1,lambda x,y:x if x else y)

In [12]:
UDEA.shape

(15726, 151)

In [13]:
UDEA[UDEA.C1==''].shape

(30, 151)

In [14]:
UDEA[UDEA.Tipo=='WOS'].reset_index(drop=True).C1.loc[0]

'[Burwick, Richard M.; Tolosa, Jorge E.] Oregon Hlth & Sci Univ, Dept Obstet & Gynecol, Div Maternal Fetal Med, Portland, OR 97201 USA.\n[Burwick, Richard M.] Cedars Sinai Med Ctr, Los Angeles, CA 90048 USA.\n[Burwick, Richard M.; Velasquez, Jesus A.; Valencia, Catalina M.; Silva, Jaime L.; Gutierrez-Marin, Jorge H.; Edna-Estrada, Francisco; Rincon, Monica; Tolosa, Jorge E.] FUNDARED MATERNA, Bogota, Colombia.\n[Velasquez, Jesus A.] Univ Antioquia, Hosp Univ San Vicente Fdn, Medellin, Colombia.\n[Velasquez, Jesus A.; Tolosa, Jorge E.] Univ Antioquia, Dept Obstet & Ginecol, Medellin, Colombia.\n[Velasquez, Jesus A.; Tolosa, Jorge E.] Univ Antioquia, NACER Salud Sexual & Reprod, Medellin, Colombia.\n[Valencia, Catalina M.] Clin Colsanitas SA, Bogota, Colombia.\n[Bernal, Yamile I.] Clin Colsanitas SA, Lab Clin Sanitas, Grp Invest INPAC, Bogota, Colombia.\n[Silva, Jaime L.] Pontificia Univ Javeriana, Hosp Univ San Ignacio, Bogota, Colombia.\n[Gutierrez-Marin, Jorge H.] Univ Pontificia Boli

In [15]:
UDEA['authors_WOS']=UDEA.C1.apply(lambda x: x.split('\n') if x else x).apply(
    lambda x:   [y.replace('[','').replace('] ','; ') for y in x if y.find(affil)>-1 ] if x else x ).apply(
     lambda x: get_author_info(x) if x else x)

# Improve normalization: remove C1s with only affiliation (from Scielo)
UDEA['authors_WOS']=UDEA['authors_WOS'].apply( 
    lambda x: [d for d in x if d.get('WOS_author').find(affil)==-1] if type(x)==list else x )

In [16]:
UDEA[UDEA.Tipo=='SCP'].reset_index(drop=True).loc[0].authors_WOS

[{'WOS_author': 'Hoyos Ossa, D. E.',
  'affiliation': ['GDCON Research GroupFaculty of EngineeringUniversity Research Headquarters (SIU)Univ Antioquia, Street 70 # 52 – 21, Medellin, Colombia'],
  'i': 0},
 {'WOS_author': 'Peñuela, G. A.',
  'affiliation': ['GDCON Research GroupFaculty of EngineeringUniversity Research Headquarters (SIU)Univ Antioquia, Street 70 # 52 – 21, Medellin, Colombia'],
  'i': 1}]

## Load trained old data 

In [17]:
if os.path.exists('WOS_SCP_UDEA_SJR_SIU.xlsx'):
    SIU=pd.read_excel('WOS_SCP_UDEA_SJR_SIU.xlsx')
else:    
    SIU=drive_files.read_drive_excel('WOS_SCP_UDEA_SJR_SIU.xlsx')

In [18]:
SIU.Tipo.unique()

array(['WOS+SCP', 'WOS+SCP+UDEA', 'WOS+SCI+SCP', 'WOS+SCI+SCP+UDEA',
       'SCI+SCP', 'SCI+SCP+UDEA', 'SCP', 'SCP+UDEA', 'WOS', 'WOS+UDEA',
       'WOS+SCI', 'WOS+SCI+UDEA', 'SCI', 'SCI+UDEA', 'UDEA'], dtype=object)

In [19]:
SIU=SIU[SIU.Tipo.str.contains('\+UDEA')].reset_index(drop=True)

In [20]:
SIU.columns.values

array(['AB', 'AF', 'AR', 'AU', 'BA', 'BE', 'BF', 'BN', 'BP', 'C1', 'CA',
       'CL', 'CR', 'CT', 'CTR_CATEGORÍA G', 'CTR_CATEGORÍA I',
       'CTR_Centro', 'CTR_CÉDULA', 'CTR_GRUPO', 'CTR_LINK CVLAC',
       'CTR_NOMBRE', 'CTR_Nivel de Formación', 'CTR_Tipo de Vinculación',
       'CTR_vinculación GrupLAC', 'CY', 'Clasificación 2016', 'D2', 'DE',
       'DI', 'DT', 'EI', 'EM', 'EP', 'FU', 'FX', 'GA', 'GP', 'HO', 'ID',
       'IS', 'ISSN', 'J9', 'JI', 'LA', 'MA', 'NR', 'OI', 'PA', 'PD', 'PG',
       'PI', 'PM', 'PN', 'PT', 'PU', 'PY', 'RI', 'RP', 'SC', 'SCI_AB',
       'SCI_AU', 'SCI_BP', 'SCI_C1', 'SCI_C2', 'SCI_CR', 'SCI_DE',
       'SCI_DI', 'SCI_DT', 'SCI_EC', 'SCI_EM', 'SCI_EP', 'SCI_IS',
       'SCI_LA', 'SCI_NR', 'SCI_OI', 'SCI_PA', 'SCI_PD', 'SCI_PI',
       'SCI_PT', 'SCI_PU', 'SCI_PY', 'SCI_RI', 'SCI_SC', 'SCI_SN',
       'SCI_SO', 'SCI_TC', 'SCI_TI', 'SCI_U1', 'SCI_U2', 'SCI_UT',
       'SCI_VL', 'SCI_X1', 'SCI_X4', 'SCI_X5', 'SCI_Y1', 'SCI_Y4',
       'SCI_Y5', 'SCI_Z1', 'S

TODO: def: Convert value string list into a list of dictionaries

In [21]:
SIU['UDEA_autores']=SIU['UDEA_autores'].str.replace('  ',' ')

In [22]:
SIU['UDEA_authors']=SIU.UDEA_autores.str.split(';').apply(lambda x: [{'full_name':y} for y in x ])

## Merge with official researcher list

In [23]:
AU=drive_files.read_drive_excel('Base_de_datos_investigadores_Definitiva.csv')

In [24]:
AU_columns=list( AU.columns.values )

In [25]:
AU['name_tmp']=(AU['PRIMER APELLIDO']+' '+AU['SEGUNDO APELLIDO']+' '+AU['NOMBRES']).str.lower().str.strip().apply( 
    unidecode.unidecode )

In [26]:
maxau=SIU['UDEA_authors'].apply(lambda x: [y.get('full_name') for y in x ]).apply(len).max()

In [27]:
kkn=SIU.copy()

In [28]:
newcolumns=['name_tmp']+AU_columns
for i in range(maxau):
    print(i)
    kkn['name_tmp']=kkn['UDEA_authors'].apply(lambda x: [y.get('full_name') for y in x ]
                            ).str[i].apply( lambda x: unidecode.unidecode( x.lower().strip()) 
                                                      if not pd.isna(x) else x)
    if not kkn[~kkn['name_tmp'].isna()].empty:
        kkn=kkn.merge(AU[newcolumns],on='name_tmp',how='left').reset_index(drop=True)
        kkn=dictionary_list_add_columns(kkn,'UDEA_authors','full_name',i,AU_columns)
        kkn=kkn.drop(newcolumns,axis='columns')

0
1
2
3
4
5
6
7
8
9


https://stackoverflow.com/a/29530601/2268280

BUG: Cells filled with nans

In [29]:
kkn.shape,SIU.shape

((7916, 205), (7916, 205))

In [30]:
SIU=kkn.copy()

In [31]:
AU[ AU['NOMBRE COMPLETO'].str.contains('Oscar Alberto Zapata Noreña') ]

,CÉDULA,NOMBRE COMPLETO,NOMBRES,PRIMER APELLIDO,SEGUNDO APELLIDO,FACULTAD,DEPARTAMENTO,GRUPO,name_tmp
234,15386534,Oscar Alberto Zapata Noreña,Oscar Alberto,Zapata,Noreña,Facultad de Ciencias Exactas y Naturales,Instituto de Física,Grupo de Fenomenologia de Interacciones Fundamentales,zapata norena oscar alberto


Quality check

In [32]:
SIU[ find_key_in_list_of_dictionaries(SIU,'UDEA_authors','full_name','ZAPATA') ].UDEA_authors.loc[241]

[{'CÉDULA': 15386534.0,
  'DEPARTAMENTO': 'Instituto de Física',
  'FACULTAD': 'Facultad de Ciencias Exactas y Naturales',
  'GRUPO': 'Grupo de Fenomenologia de Interacciones Fundamentales',
  'NOMBRE COMPLETO': 'Oscar Alberto Zapata Noreña',
  'NOMBRES': 'Oscar Alberto ',
  'PRIMER APELLIDO': 'Zapata',
  'SEGUNDO APELLIDO': 'Noreña',
  'full_name': 'ZAPATA NOREÑA OSCAR ALBERTO'},
 {'CÉDULA': 8287417.0,
  'DEPARTAMENTO': 'Instituto de Física',
  'FACULTAD': 'Facultad de Ciencias Exactas y Naturales',
  'GRUPO': 'Grupo de Fenomenologia de Interacciones Fundamentales',
  'NOMBRE COMPLETO': 'William Antonio Ponce Gutierrez',
  'NOMBRES': 'William Antonio ',
  'PRIMER APELLIDO': 'Ponce',
  'SEGUNDO APELLIDO': 'Gutierrez',
  'full_name': 'PONCE GUTIERREZ WILLIAM ANTONIO'}]

In [33]:
SIU[ find_key_in_list_of_dictionaries(SIU,'UDEA_authors','full_name','PONCE')
   ].UDEA_authors.reset_index(drop=True).loc[0]

[{'CÉDULA': 8287417.0,
  'DEPARTAMENTO': 'Instituto de Física',
  'FACULTAD': 'Facultad de Ciencias Exactas y Naturales',
  'GRUPO': 'Grupo de Fenomenologia de Interacciones Fundamentales',
  'NOMBRE COMPLETO': 'William Antonio Ponce Gutierrez',
  'NOMBRES': 'William Antonio ',
  'PRIMER APELLIDO': 'Ponce',
  'SEGUNDO APELLIDO': 'Gutierrez',
  'full_name': 'PONCE GUTIERREZ WILLIAM ANTONIO'}]

In [34]:
SIU.Tipo.unique()

array(['WOS+SCP+UDEA', 'WOS+SCI+SCP+UDEA', 'SCI+SCP+UDEA', 'SCP+UDEA',
       'WOS+UDEA', 'WOS+SCI+UDEA', 'SCI+UDEA'], dtype=object)

## Merge with trained data set

Merge requires split in DI and TI

In [36]:
#Be sure that always be True. The columns will be obtaied from SIU
REMOVE_UDEA_columns=True
if REMOVE_UDEA_columns:
    UDEA_columns=[c for c in UDEA.columns if c.find('UDEA_')>-1]
    UDEA=UDEA.drop(UDEA_columns,axis='columns')

In [37]:
udea_columns=[       'UDEA_autores',
       'UDEA_año realiz', 'UDEA_doi', 'UDEA_fecha aplicación',
       'UDEA_idioma', 'UDEA_item adic', 'UDEA_material', 'UDEA_nombre',
       'UDEA_nombre revista o premio', 'UDEA_nro autores', 'UDEA_país',
       'UDEA_procodigo', 'UDEA_ptos', 'UDEA_simple_doi', 'UDEA_título',
       'UDEA_valor item','UDEA_authors']

In [38]:
SIUDI=SIU[~SIU.DI.isna()].drop_duplicates('DI').reset_index(drop=True)
SIUTI=SIU[ SIU.DI.isna()].drop_duplicates('TI').reset_index(drop=True)
SIUTI=SIUTI[SIUTI!=''].reset_index(drop=True)
SIUTI=SIUTI[~SIUTI.TI.isnull()].reset_index(drop=True)
SIUTI=SIUTI[ SIUTI.TI.apply(len)>20 ].reset_index(drop=True)

/usr/local/lib/python3.5/dist-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [39]:
UDEADI=UDEA[UDEA.DI!=''].drop_duplicates('DI').reset_index(drop=True)
UDEATI=UDEA[UDEA.DI==''].drop_duplicates('TI').reset_index(drop=True)

In [40]:
UDEA_mergeDI=UDEADI.merge( SIUDI[ ['DI']+udea_columns ],on='DI',how='left' )

In [41]:
UDEADI.shape,UDEA_mergeDI.shape

((8035, 152), (8035, 169))

TODO: Debug

In [42]:
UDEA_PTJ=pd.DataFrame()
UDEA_PTJ_NOT=pd.DataFrame()
UDEA_PTJ=UDEA_mergeDI[~UDEA_mergeDI.UDEA_autores.isna()].reset_index(drop=True)
UDEA_PTJ_NOT=UDEA_mergeDI[UDEA_mergeDI.UDEA_autores.isna()].reset_index(drop=True)

In [43]:
UDEATI['tmptitle']=UDEATI.TI.str.strip()
SIUTI['tmptitle']=SIUTI.TI.str.strip()

In [44]:
kk=UDEATI.merge( SIUTI[ ['tmptitle']+udea_columns ],on='tmptitle',how='left' ).drop('tmptitle',axis='columns')

In [45]:
UDEA_PTJ=UDEA_PTJ.append( kk[ ~kk.UDEA_autores.isna() ] ).reset_index(drop=True)
UDEA_PTJ_NOT=UDEA_PTJ_NOT.append( kk[ kk.UDEA_autores.isna() ] ).reset_index(drop=True)

In [46]:
UDEA_PTJ.shape[0]+UDEA_PTJ_NOT.shape[0],UDEA.shape

(15722, (15726, 152))

In [47]:
UDEA_PTJ.shape,UDEA_PTJ_NOT.shape

((7074, 169), (8648, 169))

In [48]:
UDEA_PTJ.loc[[0,1]]

,AB,AF,AR,AU,BA,BE,BF,BN,BP,C1,...,UDEA_nombre,UDEA_nombre revista o premio,UDEA_nro autores,UDEA_país,UDEA_procodigo,UDEA_ptos,UDEA_simple_doi,UDEA_título,UDEA_valor item,UDEA_authors
0,,,,"Ramírez, Juan David\nGiraldo, Santiago Patiño\nArango, Marcos\n",,,,,,"[Giraldo, Santiago Patiño] Univ Antioquia, Colombia.\n[Giraldo, Santiago Patiño] Clínica CardioVID, Colombia.\n[Ramírez, Juan David; Giraldo, Santiago Patiño] Hospital Pablo Tobón Uribe, Colombia....",...,PATIÑO GIRALDO SANTIAGO,ver UDEA_material,3.0,46.0,53961.0,3.6,0.0,"LINFOMA PRIMARIO DEL CORAZON, CAUSA POCO COMUN DEL SINDROME DE FALLA CARDIACA. (PRIMARY CARDIAC LYMPHOMA, AN UNCOMMON CAUSE OF HEART FAILURE).",SOCIEDAD COLOMBIANA DE CARDIOLOGIA Y CIRUGIA CARDIOVASCULAR.,"[{'CÉDULA': 8358251.0, 'DEPARTAMENTO': 'Departamento de Medicina Interna', 'FACULTAD': 'Facultad de Medicina', 'GRUPO': 'Grupo de Investigación EDUSALUD, Informed', 'NOMBRES': 'Santiago ', 'full_..."
1,,,,"Molina, Marcela\nPalacio, Juan David\nVargas, Cristian\nDíaz-Zuluaga, Ana María\nAgudelo Berruecos, Yuli\nOspina, Sigifredo\nLópez-Jaramillo, Carlos\n",,,,,,"Hospital Universitario de San Vicente Fundación, Colombia.\n[Molina, Marcela; Palacio, Juan David; Vargas, Cristian; Díaz-Zuluaga, Ana María; Agudelo Berruecos, Yuli; Ospina, Sigifredo; López-Jara...",...,LOPEZ JARAMILLO CARLOS ALBERTO,REVISTA COLOMBIANA DE PSIQUIATRIA,7.0,46.0,55920.0,4.3,0.0,DESEMPENO NEUROCOGNITIVO DE PACIENTES CON TRASTORNO AFECTIVO BIPOLAR TIPO I EN EUTIMIA CON Y SIN ANTECEDENTE DE PSICOSIS DE UN PROGRAMA DE INTERVENCION MULTIMODAL: PRISMA,NaN,"[{'CÉDULA': 1017126043.0, 'DEPARTAMENTO': 'Departamento de Siquiatría y Ciencias del Comportamiento Hum', 'FACULTAD': 'Facultad de Medicina', 'GRUPO': 'Grupo de Investigación en Psiquiatría GIPSI'..."


In [49]:
UDEA=UDEA_PTJ.append(
    UDEA_PTJ_NOT).reset_index(
    drop=True)

In [50]:
key_contains_in_list_of_dictionaries(UDEA,'Restrepo, D',column='authors_WOS',key='WOS_author').loc[1:2]

1                                                                        [{'affiliation': ['Univ Antioquia, Inst Fis, Calle 70 52-21 Medellin, Medellin, Colombia.'], 'i': 0, 'WOS_author': 'Restrepo, D.'}]
2    [{'affiliation': ['Univ Antioquia, Inst Fis, Medellin 1226, Colombia.'], 'i': 0, 'WOS_author': 'Restrepo, Diego'}, {'affiliation': ['Univ Antioquia, Inst Fis, Medellin 1226, Colombia.'], 'i': 1, '...
Name: authors_WOS, dtype: object

In [51]:
UDEA.to_json('UDEAtmp.json')

In [52]:
RECOVER=False
if RECOVER:
    UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

## Add `UDEA.authors_WOS` info* within `UDEA.UDEA_authors` data**
(\*) obtained from `UDEA.C1`

(\*\*) Obtained from [puntaje trained old UDEA data](./WOS_SCI_SCP_PTJ_GS_LNS.ipynb#Merge-with-trained-data-set) and the [official researcher list](./WOS_SCI_SCP_PTJ_GS_LNS.ipynb#Merge-with-official-researcher-list)

Obtain name parts and initials from full name in `UDEA_authors` dictionary and update `UDEA_authors` with them

In [53]:
# tmp is for update dict
dictupdatetmp=UDEA['UDEA_authors'].apply(lambda x: [y.update( 
                split_full_names(y,full_name='full_name')  ) if not pd.isnull(
                y.get('full_name')) else y for y in x] 
                                   if type(x)==list 
                                   else x)

In [54]:
x,y=UDEA['authors_WOS'].combine( UDEA['UDEA_authors'], func=lambda x,y:(x,y) ).loc[92]

In [55]:
x

[{'WOS_author': 'Echeverri Alvarez, Jonathan',
  'affiliation': ['Univ Antioquia, Antioquia, Colombia.'],
  'i': 0},
 {'WOS_author': 'Chaves Castano, Liliana',
  'affiliation': ['Univ Antioquia, Antioquia, Colombia.'],
  'i': 1}]

In [56]:
y

[{'CÉDULA': 43724493.0,
  'DEPARTAMENTO': 'Departamento de Sicología',
  'FACULTAD': 'Facultad de Ciencias Sociales y Humanas',
  'GRUPO': 'Grupo de investigación en psicologia cognitiva',
  'INICIALES': 'L.',
  'NOMBRE COMPLETO': 'Liliana Chaves Castaño',
  'NOMBRES': 'Liliana',
  'PRIMER APELLIDO': 'Chaves',
  'SEGUNDO APELLIDO': 'Castaño',
  'full_name': 'CHAVES CASTAÑO LILIANA'}]

In [57]:
def wos_names_append(wos_names,last_name,first_names,initials):
    wos_names=wos_names+[           last_name+', '+first_names]
    if len( first_names.split())>1:
        wos_names=wos_names+[ last_name+', '+first_names.split()[0] ]
    if len( first_names.split())==2 and  len(first_names.split()[-1]):
        wos_names=wos_names+[ last_name+', '+first_names.split()[-1] ]
    wos_names=wos_names+[ last_name+', '+initials]
    if len( initials.split())>1:
        wos_names=wos_names+[ last_name+', '+initials.split()[0]]
    if len(initials.split())==2:
        wos_names=wos_names+[
              last_name+', '+first_names.split()[0]+' '+initials.split()[-1] ]
        wos_names=wos_names+[last_name+', '+initials.split()[-1] ]
    return wos_names
    
def wos_names_list(dy ,y_keys=['PRIMER APELLIDO','NOMBRES','INICIALES','SEGUNDO APELLIDO','full_name']   ):
    last_name= unidecode.unidecode( dy[y_keys[0]]   )
    first_names=unidecode.unidecode( dy[y_keys[1]]  )
    initials=unidecode.unidecode( dy[y_keys[2]]  )

    wos_names=[]
    wos_names=wos_names_append(wos_names,last_name,first_names,initials)
    
    if dy.get( y_keys[3] ):
        last_names= unidecode.unidecode( dy[y_keys[0]]+'-'+dy[y_keys[3]]   )
        wos_names=wos_names_append(wos_names,last_names,first_names,initials)
    return wos_names
    
def combinewos(x,y,x_keys=['WOS_author','affiliation'],
                   y_keys=['PRIMER APELLIDO','NOMBRES','INICIALES','SEGUNDO APELLIDO','full_name'],
                   xy_keys=['WOS_author','WOS_affiliation']):
    if type(x)==list and type(y)==list:
        for dx in x:
            wos_name=unidecode.unidecode( dx[ x_keys[0] ] )
            WOS_affiliation= dx[x_keys[1]]
            # Try by buildinng spanish-like names list                                
            for i in range( len(y) ):
                if wos_name.title() in wos_names_list(y[i] ,y_keys):
                    y[i][  xy_keys[0] ]=[ wos_name ]
                    y[i][  xy_keys[1] ]=WOS_affiliation
                    break
            #Try again but comparing full lists
            wos_name_to_list=wos_name.replace(',','').replace('-',' ').title().split()
            for i in range( len(y) ):
                yi_to_list=unidecode.unidecode( y[i][y_keys[4]].title() )
                if yi_to_list:
                    yi_to_list=yi_to_list.split()
                else:
                    yi_to_list=[]
                if not pd.np.setdiff1d(wos_name_to_list,yi_to_list).shape[0]:
                    y[i][  xy_keys[0] ]=[ wos_name ]
                    y[i][  xy_keys[1] ]=WOS_affiliation
                    break
            #Try again but comparing full lists with initials
            wos_name_to_list=wos_name.replace(',','').replace('-',' ').title().split()
            for i in range( len(y) ):
                yi_to_list=[y[i][y_keys[0]],y[i][y_keys[3]] ]+y[i][y_keys[2]].split()
                if not pd.np.setdiff1d(wos_name_to_list,yi_to_list).shape[0]:
                    y[i][  xy_keys[0] ]=[ wos_name ]
                    y[i][  xy_keys[1] ]=WOS_affiliation
                    break
            #Try again but comparing full lists with first first name and initial
            for i in range( len(y) ):
                yi_to_list=[y[i][y_keys[0]],y[i][y_keys[3]],y[i][y_keys[1]].split()[0],
                              y[i][y_keys[2]].split()[-1]]
                if not pd.np.setdiff1d(wos_name_to_list,yi_to_list).shape[0]:
                    y[i][  xy_keys[0] ]=[ wos_name ]
                    y[i][  xy_keys[1] ]=WOS_affiliation
                    break                    
            #Try again but comparing full lists with second first name and initial
            for i in range( len(y) ):
                yi_to_list=[y[i][y_keys[0]],y[i][y_keys[3]],y[i][y_keys[1]].split()[-1],
                              y[i][y_keys[2]].split()[0]]
                if not pd.np.setdiff1d(wos_name_to_list,yi_to_list).shape[0]:
                    y[i][  xy_keys[0] ]=[ wos_name ]
                    y[i][  xy_keys[1] ]=WOS_affiliation
                    break                    
                    
                    
    return y

In [58]:
wos_names_list( split_full_names( y[0] ) )

['Chaves, Liliana',
 'Chaves, L.',
 'Chaves-Castano, Liliana',
 'Chaves-Castano, L.']

In [59]:
#kk=UDEA['UDEA_authors'].str[0].apply( lambda d: d if type(d)==dict else pd.np.nan ).dropna()
#kk[kk.apply( lambda d: d.get('full_name') if type(d)==dict else pd.np.nan ).isna()]

In [60]:
kk=UDEA['authors_WOS'].combine( UDEA['UDEA_authors'], func=combinewos )

Combines the two list into a single one in  UDEA_authors

check

Full query of auhor_WOS in UDEA_authors:

In [61]:
UDEA['UDEA_authors'].apply(lambda x: [y  for y in x ][0] if type(x)==list else x).loc[92]

{'CÉDULA': 43724493.0,
 'DEPARTAMENTO': 'Departamento de Sicología',
 'FACULTAD': 'Facultad de Ciencias Sociales y Humanas',
 'GRUPO': 'Grupo de investigación en psicologia cognitiva',
 'INICIALES': 'L.',
 'NOMBRE COMPLETO': 'Liliana Chaves Castaño',
 'NOMBRES': 'Liliana',
 'PRIMER APELLIDO': 'Chaves',
 'SEGUNDO APELLIDO': 'Castaño',
 'WOS_affiliation': ['Univ Antioquia, Antioquia, Colombia.'],
 'WOS_author': ['Chaves Castano, Liliana'],
 'full_name': 'CHAVES CASTAÑO LILIANA'}

In [62]:
UDEA.to_json('UDEAtmp.json')

### Load output restuls of previous Cell runs

In [63]:
RECOVER=False
if RECOVER:
    UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

In [64]:
def extract_internal_value_of_a_dictionary_key_in_a_list_of_dictionaries(df,
    list_of_dictionaries='UDEA_authors',
    dictionary_key='WOS_author'):
    #Extract internal value of a dictionary key in a list of dictionaries and empty list otherwise
    
    return df[list_of_dictionaries].apply(lambda x: [y.get(dictionary_key) 
                                                        if y.get(dictionary_key) else []   
                                                        for y in x  ]
                           if type(x)==list else [])

def extract_internal_list_as_value_of_a_dictionary_key_in_a_list_of_dictionaries(df,
    list_of_dictionaries='UDEA_authors',
    dictionary_key='WOS_author'):
    #Extract internal list as value of a dictionary key in a list of dictionaries and empty list otherwise
    
    return extract_internal_value_of_a_dictionary_key_in_a_list_of_dictionaries(df,
                    list_of_dictionaries,dictionary_key).apply(lambda x: 
                           [item for sublist  in x for item in sublist] 
                            if type(x)==list else x)

def mask_on_internal_value_of_a_dictionary_key_in_a_list_of_dictionaries(df,
    pattern='RESTREPO QUINTERO DIEGO ALEJANDRO',
    list_of_dictionaries='UDEA_authors',dictionary_key='full_name'):
    """
    Build a mask for a Pandas Series of list of dictionaries of label:
      list_of_dictionaries. 
    The:
      dictionary_key must be a single value like string or float
    """
    return extract_internal_value_of_a_dictionary_key_in_a_list_of_dictionaries(df,
            list_of_dictionaries,dictionary_key).apply( 
            lambda x: pattern in x)

In [65]:
pattern='Restrepo, Diego'
mask=extract_internal_list_as_value_of_a_dictionary_key_in_a_list_of_dictionaries(
            UDEA,list_of_dictionaries='UDEA_authors',dictionary_key='WOS_author').apply( 
            lambda x: pattern in x)
UDEA[mask]['UDEA_authors'].reset_index(drop=True)

0     [{'CÉDULA': 98554575.0, 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interacciones Fundamentales', 'WOS_affiliation': ['Univ Antioquia, Inst Fis, Calle 70 52-21, Mede...
1     [{'CÉDULA': 98554575.0, 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interacciones Fundamentales', 'WOS_affiliation': ['Univ Antioquia, Inst Fis, Calle 70 52-21, Mede...
2     [{'CÉDULA': 98554575.0, 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interacciones Fundamentales', 'WOS_affiliation': ['Univ Antioquia, Inst Fis, Calle 70 52-21, Mede...
3     [{'CÉDULA': 15386534.0, 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interacciones Fundamentales', 'WOS_affiliation': ['Univ Antioquia, Inst Fis, Medellin, Colombia.'...
4     [{'CÉDULA': 15386534.0, 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interacciones Fundamentales', 'WOS_affiliation': ['Univ Antioquia, Inst

In [66]:
pattern='Restrepo, D.'
mask=extract_internal_list_as_value_of_a_dictionary_key_in_a_list_of_dictionaries(
            UDEA,list_of_dictionaries='UDEA_authors',dictionary_key='WOS_author').apply( 
            lambda x: pattern in x)
UDEA[mask]['UDEA_authors'].reset_index(drop=True)

0     [{'CÉDULA': 98554575.0, 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interacciones Fundamentales', 'WOS_affiliation': ['Univ Antioquia, Inst Fis, Calle 70 52-21 Medel...
1     [{'CÉDULA': 98554575.0, 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interacciones Fundamentales', 'WOS_affiliation': ['Univ Antioquia, Inst Fis, Medellin, Colombia.'...
2     [{'CÉDULA': 98554575.0, 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interacciones Fundamentales', 'WOS_affiliation': ['Univ Antioquia, Inst Fis, Medellin 1226, Colom...
3     [{'CÉDULA': 98554575.0, 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interacciones Fundamentales', 'WOS_affiliation': ['Univ Antioquia, Inst Fis, Medellin 1226, Colom...
4     [{'CÉDULA': 98554575.0, 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interacciones Fundamentales', 'WOS_affiliation': ['Univ Antioquia, Inst

In [67]:
pattern

'Restrepo, D.'

In [68]:
mask=extract_internal_list_as_value_of_a_dictionary_key_in_a_list_of_dictionaries(
            UDEA,list_of_dictionaries='UDEA_authors',dictionary_key='WOS_author').apply( 
            lambda x: pattern in x)
UDEA[mask]['UDEA_authors'].reset_index(drop=True)

0     [{'CÉDULA': 98554575.0, 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interacciones Fundamentales', 'WOS_affiliation': ['Univ Antioquia, Inst Fis, Calle 70 52-21 Medel...
1     [{'CÉDULA': 98554575.0, 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interacciones Fundamentales', 'WOS_affiliation': ['Univ Antioquia, Inst Fis, Medellin, Colombia.'...
2     [{'CÉDULA': 98554575.0, 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interacciones Fundamentales', 'WOS_affiliation': ['Univ Antioquia, Inst Fis, Medellin 1226, Colom...
3     [{'CÉDULA': 98554575.0, 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interacciones Fundamentales', 'WOS_affiliation': ['Univ Antioquia, Inst Fis, Medellin 1226, Colom...
4     [{'CÉDULA': 98554575.0, 'DEPARTAMENTO': 'Instituto de Física', 'GRUPO': 'Grupo de Fenomenologia de Interacciones Fundamentales', 'WOS_affiliation': ['Univ Antioquia, Inst

In [69]:
def extract_dictionary_from_list(df,list_name,i=0):
    return df[list_name].apply( lambda l: [ d for d in l][i] if type(l)==list else l )
def extract_key_value_from_series(ds,key):
    return ds.apply( lambda d: d.get(key)if type(d)==dict else d  )

In [70]:
mask=extract_key_value_from_series( 
    extract_dictionary_from_list( UDEA,'UDEA_authors',i=0),'WOS_author').str[0].str.contains('\-').fillna(False)
UDEA[mask]['UDEA_authors'].reset_index(drop=True).str[0].loc[1]

{'CÉDULA': 87470374.0,
 'DEPARTAMENTO': 'Departamento de Microbiología y Parasitología',
 'FACULTAD': 'Facultad de Medicina',
 'GRUPO': 'Inmunovirología',
 'INICIALES': 'S.',
 'NOMBRE COMPLETO': 'Silvio Urcuqui Inchima',
 'NOMBRES': 'Silvio',
 'PRIMER APELLIDO': 'Urcuqui',
 'SEGUNDO APELLIDO': 'Inchima',
 'WOS_affiliation': ['Univ Antioquia, Sede Invest Univ, Grp Inmunoviol, Medellin, Colombia.'],
 'WOS_author': ['Urcuqui-Inchima, Silvio'],
 'full_name': 'URCUQUI INCHIMA SILVIO'}

TODO: Extract dict or value

## Build a single profile for all

flatten a list of lists:

In [71]:
ua=pd.DataFrame()

In [72]:
aumax=UDEA['UDEA_authors'].dropna().apply(len).max() 
ua=pd.DataFrame()
for i in range(aumax):
    kkk=pd.DataFrame()
    kkk['UDEA_authors']= UDEA['UDEA_authors'].str[i].dropna()
    kkk['authors_WOS']= UDEA['authors_WOS']
    kkk['SCP_Authors']=UDEA['SCP_Authors']
    kkk['tmp_str']=kkk['UDEA_authors'].astype(str)
    kkk=kkk.drop_duplicates('tmp_str')
    ua=ua.append(kkk).reset_index(drop=True)
    #ua['UDEA_authors']= 

In [73]:
ua['tmp_author']=ua['UDEA_authors'].apply( lambda d: d.get('full_name') )

BUG: `UDEA_authors` without `WOS_author key`

### Filter identified WOS articles

In [74]:
ua['authors_WOS']=ua['authors_WOS'].apply(lambda l: l if l else pd.np.nan)
ua=ua[~ua['authors_WOS'].isna()].reset_index(drop=True)

In [75]:
full_names=ua['tmp_author'].unique()

In [76]:
full_names[:3]

array(['PATIÑO GIRALDO SANTIAGO', 'VARGAS UPEGUI CRISTIAN DAVID',
       'AMARILES MUÑOZ PEDRO JOSE'], dtype=object)

In [77]:
ua.shape,full_names.shape

((7181, 5), (1433,))

TODO: implement also SCP authors

Example

In [78]:
aunly=pd.DataFrame()
for f in full_names:
    clear_output(wait=True)
    print(f)    
    kk=pd.DataFrame( { 'tmp_author':[f]  } ).merge(
          ua[['tmp_author','UDEA_authors']],on='tmp_author',how='left')

    kk['tmp_str']=kk.UDEA_authors.astype(str)

    kk=kk.drop_duplicates('tmp_str').dropna()#[['tmp_author','UDEA_authors']]

    try:
        laff=list( kk['UDEA_authors'].apply(lambda d: d.get( 'WOS_affiliation' )
                                     ).dropna().apply(pd.Series).stack().unique() )
        lau=list( kk['UDEA_authors'].apply(lambda d: d.get( 'WOS_author' )
                                     ).dropna().apply(pd.Series).stack().unique() )
    except AttributeError:
        laff=[];lau=[]

    if len(laff)>0 and len(lau)>0:
        tmpupdate=kk['UDEA_authors'].apply(lambda d: d.update({'WOS_author':lau,'WOS_affiliation':laff}) )

        kk['tmp_str']=kk['UDEA_authors'].astype(str)

        kk=kk.drop_duplicates('tmp_str')

        kk['tmp_len']=kk['tmp_str'].apply(len)#.astype(str)

        aunly=aunly.append( kk.sort_values('tmp_len',ascending=False).drop(index=kk.index[1:]).drop(
               ['tmp_str','tmp_len'],axis='columns') ).reset_index(drop=True)

AGUIRRE GAVIRIA BEATRIZ EUGENIA


In [79]:
#DEBUG
aunly[aunly.tmp_author.fillna('').str.contains('DUQUE ECHEVERRI CARLOS ALBERTO')
      ].reset_index(drop=True).UDEA_authors.loc[0]

{'CÉDULA': 71617883.0,
 'DEPARTAMENTO': 'Instituto de Física',
 'FACULTAD': 'Facultad de Ciencias Exactas y Naturales',
 'GRUPO': 'Grupo de Materia Condensada-UdeA',
 'INICIALES': 'C. A.',
 'NOMBRE COMPLETO': 'Carlos Alberto Duque Echeverri',
 'NOMBRES': 'Carlos Alberto',
 'PRIMER APELLIDO': 'Duque',
 'SEGUNDO APELLIDO': 'Echeverri',
 'WOS_affiliation': ['Univ Antioquia UdeA, Fac Ciencias Exactas & Nat, Inst Fis, Grp Mat Condensada UdeA, Medellin, Colombia.',
  'Univ Antioquia UdeA, Grp Materia Condensada UdeA, Inst Fis, Fac Ciencias Exactas & Nat, Medellin, Colombia.',
  'Univ Antioquia UdeA, Inst Fis, Fac Ciencias Exactas & Nat, Grp Mat Condensada UdeA, Medellin, Colombia.',
  'Instituto de Física, Univ Antioquia, A.A. 1226, Medellín, Colombia',
  'Instituto de Física, Univ Antioquia, AA 1226, Medellín, Colombia',
  'Univ Antioquia UdeA, Fac Ciencias Exactas & Nat, Inst Fis, Grp Mat Condensada UdeA, Calle 70 52-21, Medellin, Colombia.',
  'Univ Antioquia UdeA, Inst Fis, Fac Ciencias 

In [80]:
ua['UDEA_authors'].loc[0]

{'CÉDULA': 8358251.0,
 'DEPARTAMENTO': 'Departamento de Medicina Interna',
 'FACULTAD': 'Facultad de Medicina',
 'GRUPO': 'Grupo de Investigación EDUSALUD, Informed',
 'INICIALES': 'S.',
 'NOMBRE COMPLETO': 'Santiago Patiño Giraldo',
 'NOMBRES': 'Santiago',
 'PRIMER APELLIDO': 'Patiño',
 'SEGUNDO APELLIDO': 'Giraldo',
 'WOS_affiliation': ['Univ Antioquia, Colombia.'],
 'WOS_author': ['Giraldo, Santiago Patino'],
 'full_name': 'PATIÑO GIRALDO SANTIAGO'}

In [81]:
ua['authors_WOS'].loc[0]

[{'WOS_author': 'Giraldo, Santiago Patiño',
  'affiliation': ['Univ Antioquia, Colombia.'],
  'i': 0}]

In [82]:
aunly.shape

(1273, 2)

In [83]:
aunly.to_json('UDEA_authors_with_WOS_info.json')

In [84]:
RECOVER=False
if RECOVER:
    UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

In [95]:
UDEA.shape

(15722, 169)

In [86]:
if RECOVER:
    if os.path.exists('UDEA_authors_with_WOS_info.json' ):
        aunly=pd.read_json('UDEA_authors_with_WOS_info.json')
    else:
        aunly=drive_files.read_drive_json('UDEA_authors_with_WOS_info.json')

In [87]:
aunly.shape

(1273, 2)

(800, 2)

## Merge UDEA with authors

In [88]:
# loop upon i and j to merge aunly in UDEA such that the full authors and affiliations are registered
#UDEA.merge

In [89]:
def fill_full_wos_author_info(l,WOS_df=aunly,full_name='full_name',full_name_column='tmp_author',
                               WOS_column='UDEA_authors',WOS_author='WOS_author',
                               WOS_affiliation='WOS_affiliation'):
    newl=[]
    if type(l)==list:
        for d in l:
            if d.get('WOS_author'):
                #find in aunly
                mtch=WOS_df[WOS_df[full_name_column]==d.get(full_name)].reset_index(drop=True)
                if mtch.shape[0]==1:
                    #update d
                    if mtch[WOS_column].loc[0].get(WOS_author):
                        d[WOS_author]=mtch[WOS_column].loc[0].get(WOS_author)
                        d[WOS_affiliation]=mtch[WOS_column].loc[0].get(WOS_affiliation)
            newl.append(d)
    else:
        newl=l
    return newl        

In [90]:
#l=UDEA['UDEA_authors'].loc[4]
#fill_full_wos_author_info(l,aunly)[0].get('WOS_author')

In [91]:
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply(fill_full_wos_author_info )

In [92]:
UDEA.to_json('UDEAtmp.json')

In [93]:
RECOVER=True
if RECOVER:
    UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

In [94]:
UDEA.shape

(15722, 169)

## Implementation without merge

In [96]:
import Levenshtein as lv
def find_author_affiliation(author,affiliation,author_df=aunly,column='UDEA_authors',
                            author_key='WOS_author',affiliation_key='WOS_affiliation',ratio=0.9):
    '''
    find the WOS+"UDEA puntaje" dictionary for WOS author:   `author`
    and WOS affiliation:                                    `affiliation`
    The information is  searched in 
    WOS+"UDEA puntaje" DataFrame:                            `author_df`, 
    which has the column:                                    `column` 
    which contains a dictionary with list value for the key: `author_key`
    and list value for the key:                              `affiliation_key`.
    Affiliation must be similar until a Levenshtein ratio:   `ratio`
    '''    
    au=author_df[ author_df[column].apply(
                lambda d: d.get(author_key) if type(d)==dict else '').apply(
                  lambda l: author in l)]#.reset_index(drop=True).loc[0,column]
    if au.shape[0]>0:
        #Fast
        auf=au[au[column].apply(
                 lambda d: d.get(affiliation_key) if type(d)==dict else '').apply(
                 lambda l: affiliation in l)]
        

        if auf.shape[0]>0:
              return auf.reset_index(drop=True).loc[0,column]          
        #Slow
        else:
            aus=au[au[column].apply(
                 lambda d: d.get(affiliation_key) if type(d)==dict else '').apply(
                 lambda l: len( [af for af in l if lv.ratio(af,affiliation) > ratio ] )>0 )]

            if aus.shape[0]==1: #fix 1 to avoid homonymous
                dold=aus.reset_index(drop=True).loc[0,column]
                # Dictionary is automatically updated in author_df!
                dold[affiliation_key]=dold[affiliation_key]+[affiliation]
                return dold
    else:
        return None

def get_UDEA_authors(x,y,x_author_key='WOS_author',x_affiliation_key='affiliation',
                        author_df=aunly,column='UDEA_authors',
                        author_key='WOS_author',affiliation_key='WOS_affiliation',
                        ratio=0.9):
    '''
    get the WOS+"UDEA puntaje" list of dictionaries for WOS author list 
    and affiliation list in the  list of dictionaries:      `x`, 
    where each dictionary have the string value for the key: `x_author_key`, 
    and the list value for the key:                          `x_affiliation_key`.
    The information is obtained directly from the 
    WOS+"UDEA puntaje" list:                                 `y` 
    if already there, or searched in 
    WOS+"UDEA puntaje" DataFrame:                            `author_df`, 
    which has the column:                                    `column` 
    which contains a dictionary with list value for the key: `author_key`
    and list value for the key:                              `affiliation_key`.
    If not foun None is returned.
    WOS info can be changed for other standarized dababase info
    and "UDEA puntaje" can be changed from any other full name author
    and affiliation info.
    
    IMPORTANT:
    The list of values in:                                   `affiliation_key` 
    is automatically updated with the similar first 
    affiliation value of the list in:                        `x_affiliation_key`
    according with the Levenshtein similarity ratio:         `ratio`
    '''
    if type(y)==list:
        #already filled:
        return y

    au=[]
    if ( type(x)==list and x):
        for j in range(len(x)):
            xx=find_author_affiliation(x[j].get(x_author_key),x[j].get(x_affiliation_key)[0],
                                        author_df=author_df,
                                        author_key=author_key,
                                        affiliation_key=affiliation_key,
                                        ratio=ratio )
            if xx:
                au.append(xx)
    if au:
        return au
    else:
        return None

In [97]:
i=6
x=UDEA['authors_WOS'].loc[i]
y=UDEA['UDEA_authors'].loc[i]

In [98]:
x

[{'WOS_author': 'Jennifer Moreno, Diana',
  'affiliation': ['Univ Antioquia, Grp Genet Mol, Medellin, Colombia.'],
  'i': 0},
 {'WOS_author': 'Ruiz, Susana',
  'affiliation': ['Univ Antioquia, Grp Genet Mol, Medellin, Colombia.'],
  'i': 1},
 {'WOS_author': 'Bedoya, Gabriel',
  'affiliation': ['Univ Antioquia, Grp Genet Mol, Medellin, Colombia.'],
  'i': 2},
 {'WOS_author': 'Lopera, Francisco',
  'affiliation': ['Univ Antioquia, Grp Neurociencias, Medellin, Colombia.'],
  'i': 3}]

In [99]:
y

In [100]:
get_UDEA_authors(x,y)

[{'CÉDULA': 8342412.0,
  'DEPARTAMENTO': 'Instituto de Biología',
  'FACULTAD': 'Facultad de Ciencias Exactas y Naturales',
  'GRUPO': 'Genética Molecular (GENMOL)',
  'INICIALES': 'G. J.',
  'NOMBRE COMPLETO': 'Gabriel De Jesus Bedoya Berrio',
  'NOMBRES': 'Gabriel Jesus',
  'PRIMER APELLIDO': 'Bedoya',
  'SEGUNDO APELLIDO': 'Berrio',
  'WOS_affiliation': ['Laboratorio de Genetica Molecular, Univ Antioquia, Medellin, Colombia',
   'Univ Antioquia, Inst Biol, Mol Genet Lab, Medellin, Colombia.',
   'Univ Antioquia UdeA, Mol Genet Grp, Calle 70 52-21, Medellin, Colombia.',
   'Univ Antioquia, UdeA, Medellin, Colombia.',
   'Univ Antioquia, Inst Biol, Lab Mol Genet, Medellin, Colombia.',
   'Univ Antioquia, GENMOL, Medellin 1226, Colombia.',
   'Univ Antioquia, Genet Mol Lab, SIU, Medellin, Colombia.',
   'Univ Antioquia, Lab Genet Mol, Medellin, Colombia.',
   'Univ Antioquia, Genet Mol Lab, Medellin, Colombia.',
   'Univ Antioquia, Inst Biol, Genet Mol Lab, Medellin, Colombia.',
   'Un

In [101]:
kk=UDEA.authors_WOS.combine(UDEA.UDEA_authors,func=get_UDEA_authors)

In [104]:
UDEA.UDEA_authors.dropna().shape

(7074,)

In [105]:
UDEA['UDEA_authors']=kk

In [106]:
UDEA.UDEA_authors.dropna().shape

(8609,)

In [107]:
aunly.to_json('UDEA_authors_with_WOS_info.json')

In [108]:
RECOVER=True
if RECOVER:
    if os.path.exists('UDEA_authors_with_WOS_info.json' ):
        aunly=pd.read_json('UDEA_authors_with_WOS_info.json')
    else:
        aunly=drive_files.read_drive_json('UDEA_authors_with_WOS_info.json')

In [109]:
UDEA.to_json('UDEAtmp.json')

In [110]:
if RECOVER:
    UDEA=pd.read_json('UDEAtmp.json').reset_index(drop=True)

### Test

In [203]:
qq=UDEA.copy()
qq['UDEA_authors']=None

In [207]:
qqq=qq.authors_WOS.combine(qq.UDEA_authors,func=get_UDEA_authors)

In [208]:
qq=UDEA.copy()
qq['NEW']=qqq
qq['EXTRA']=kk

In [209]:
(qq[qq.UDEA_authors.isna()][['NEW','EXTRA']].dropna(subset=['NEW']))[qq.EXTRA==None].shape

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(0, 2)

# TMP

The entries left are the ones where there are normalization problemas
* Bad written names
* Wrong affiliation in WOS
* Author appears in AU but not in C1

In [ ]:
i=0
tmpua=UDEA[UDEA.UDEA_authors.str[i].apply(lambda d: True if type(d)==dict else False)]

In [ ]:
tmpkk=tmpua[ tmpua.UDEA_authors.str[i].apply(lambda d: d.get('WOS_author')).str[0].apply( pd.isna) ]

In [ ]:
tmp=tmpkk[ tmpkk.authors_WOS.apply( lambda l: l if l and type(l)==list else pd.np.nan ).apply( 
    lambda l: True if type(l)==list else False) ][['authors_WOS','UDEA_authors','UDEA_autores','TI','DI','AU','C1']].reset_index(drop=True)#.loc[4].values

In [ ]:
tmp.shape

In [ ]:
i=204
tmp[i:i+1]#.UDEA_authors#.loc[0]

In [ ]:
tmp.UDEA_authors.loc[i]

In [ ]:
tmp.authors_WOS.loc[i]

In [ ]:
tmp.UDEA_autores.loc[i]

In [ ]:
tmp.TI.loc[i]

In [ ]:
print(tmp.AU.loc[i])

In [ ]:
print(tmp.C1.loc[i])

In [ ]:
x=tmp['authors_WOS'].reset_index(drop=True).loc[i]
y=tmp['UDEA_authors'].reset_index(drop=True).loc[i]

In [ ]:
x

In [ ]:
y

In [ ]:
i=0
y#[i]['NOMBRE COMPLETO'].split()

In [ ]:
y_keys=['PRIMER APELLIDO','NOMBRES','INICIALES','SEGUNDO APELLIDO','full_name']

In [ ]:
i=0
wos_names_list(y[i] ,y_keys)

In [ ]:
x

In [ ]:
[1,
  2]

In [ ]:
def combinewos(x,y,x_keys=['WOS_author','affiliation'],
                   y_keys=['PRIMER APELLIDO','NOMBRES','INICIALES','SEGUNDO APELLIDO','full_name'],
                   xy_keys=['WOS_author','WOS_affiliation']):
    if type(x)==list and type(y)==list:
        for dx in x:
            print( dx[ x_keys[0] ] )
            wos_name=unidecode.unidecode( dx[ x_keys[0] ] )
            WOS_affiliation= dx[x_keys[1]]
            print('Name True')
            for i in range( len(y) ):
                print('+'*20)
                print( wos_names_list(y[i] ,y_keys))
                print('+'*20)
                print(i,wos_name in wos_names_list(y[i] ,y_keys) )
                print('='*20)
                if wos_name.title() in wos_names_list(y[i] ,y_keys):
                    print('*'*20)
                    print(wos_name in wos_names_list(y[i] ,y_keys),':',wos_name)
                    print('*'*20)
                    y[i][  xy_keys[0] ]=[ wos_name ]
                    y[i][  xy_keys[1] ]=WOS_affiliation
                    break
            # Try by buildong spanish-like names list                    
            #Try again but comparing full lists
            wos_name_to_list=wos_name.replace(',','').replace('-',' ').title().split()
            print("-"*20)
            for i in range( len(y) ):
                print('--',i)
                yi_to_list=unidecode.unidecode( y[i][y_keys[4]].title() )
                if yi_to_list:
                    yi_to_list=yi_to_list.split()
                else:
                    yi_to_list=[]
                print(i,wos_name_to_list,yi_to_list,  )    
                if not pd.np.setdiff1d(wos_name_to_list,yi_to_list).shape[0]:
                    y[i][  xy_keys[0] ]=[ wos_name ]
                    y[i][  xy_keys[1] ]=WOS_affiliation
                    break
            #Try again but comparing full lists with initials
            wos_name_to_list=wos_name.replace(',','').replace('-',' ').title().split()
            for i in range( len(y) ):
                yi_to_list=[y[i][y_keys[0]],y[i][y_keys[3]] ]+y[i][y_keys[2]].split()
                if not pd.np.setdiff1d(wos_name_to_list,yi_to_list).shape[0]:
                    y[i][  xy_keys[0] ]=[ wos_name ]
                    y[i][  xy_keys[1] ]=WOS_affiliation
                    break
            #Try again but comparing full lists with first first name and initial
            print(":"*20)
            for i in range( len(y) ):
                yi_to_list=[y[i][y_keys[0]],y[i][y_keys[3]],y[i][y_keys[1]].split()[0],
                              y[i][y_keys[2]].split()[-1]]
                print("::::",yi_to_list)
                if not pd.np.setdiff1d(wos_name_to_list,yi_to_list).shape[0]:
                    y[i][  xy_keys[0] ]=[ wos_name ]
                    y[i][  xy_keys[1] ]=WOS_affiliation
                    break                    
            #Try again but comparing full lists with second first name and initial
            print(":"*20)
            for i in range( len(y) ):
                yi_to_list=[y[i][y_keys[0]],y[i][y_keys[3]],y[i][y_keys[1]].split()[-1],
                              y[i][y_keys[2]].split()[0]]
                print("::::",yi_to_list)
                if not pd.np.setdiff1d(wos_name_to_list,yi_to_list).shape[0]:
                    y[i][  xy_keys[0] ]=[ wos_name ]
                    y[i][  xy_keys[1] ]=WOS_affiliation
                    break                    
                    
                    
                    
    return y

In [ ]:
combinewos(x,y)

In [ ]:
pd.np.setdiff1d(['Bastidas', 'Myriam'] ,['Bastidas', 'Acevedo', 'Miryam', 'Del', 'Socorro']).shape[0]

In [ ]:
800*3*3

In [ ]:
aunly

## TODO: Try to merge new WOS articles from old `'UDEA_authors'`

TODO: whithout split or merge

Split DataFrame into  "new" with UDEA authors not yet identified and "old" with full UDEA author info 

In [ ]:
UDEAnew=UDEA[UDEA.UDEA_authors.isna()].reset_index(drop=True)
UDEAold=UDEA[~UDEA.UDEA_authors.isna()].reset_index(drop=True)

### Merge old with news

Split "new" into the "Y" with WOS author info

In [ ]:
UDEAnewY=UDEAnew[ UDEAnew.authors_WOS.apply(lambda x: type(x)==list and len(x)>0) ].reset_index(drop=True)
UDEAnewN=UDEAnew[~UDEAnew.authors_WOS.apply(lambda x: type(x)==list and len(x)>0) ].reset_index(drop=True)

In [ ]:
(UDEAnewY.shape,'+',UDEAnewN.shape,'+',UDEAold.shape,'=',
   UDEAnewY.shape[0]+UDEAnewN.shape[0]+UDEAold.shape[0],'=',UDEA.shape[0])

In [ ]:
CHECK=False
if CHECK:
    pattern='Restrepo, D.'
    mask=extract_internal_value_of_a_dictionary_key_in_a_list_of_dictionaries(
                UDEAnewY,list_of_dictionaries='authors_WOS',dictionary_key='WOS_author').apply( 
                lambda x: pattern in x)
    UDEAnewY[mask].authors_WOS.reset_index(drop=True).values

In [ ]:
if CHECK:
    mask=mask_on_internal_value_of_a_dictionary_key_in_a_list_of_dictionaries(UDEAnewY,
            pattern='Restrepo, D.',list_of_dictionaries='authors_WOS',dictionary_key='WOS_author')                                                                       
    UDEAnewY[mask]['authors_WOS'].reset_index(drop=True).values

In [ ]:
#Extract a specific author with specific affiliation from the WOS author full info into a temporal column
i=0
aff_i=0
UDEAnewY['news_i_j_str']=UDEAnewY.authors_WOS.apply(lambda x: [ {'WOS_author':y.get('WOS_author'),
                                                   'affiliation':y.get('affiliation')[aff_i]} for y in x]
                                                   ).str[i].astype(str)
UDEAnewY['news_i_j_str'].loc[0]

Prepare trained data set with the j-th author: first UDEA author info into a temporal column

In [ ]:
j=0
auth_j=0
aff_j=0
UDEAold['UDEA_authors_j']=UDEAold['UDEA_authors'].str[j]

In [ ]:
#Extract a specific author with specific affiliation from the WOS author full info into a temporal column
UDEAold['news_i_j']=UDEAold['UDEA_authors'].apply(lambda l: [ {'WOS_author':d.get('WOS_author')[auth_j], 
                                           'affiliation':d.get('WOS_affiliation')[aff_j] } if d.get('WOS_author') 
                                         else pd.np.nan for d in l] ).str[j]#.astype(str)


In [ ]:
ma=UDEAold.dropna(subset=['news_i_j']).reset_index(drop=True)

### Quality critera search:
* Affiliation split(', ') > 3
* if Author find('.') → 'Author split(' ')  > 2 

In [ ]:
ma['news_i_j'].loc[2]

In [ ]:
#*** May be filter Homonymous
Homonymous=False
if Homonymous:
    ma=ma[~np.logical_and( ma['news_i_j'].apply(lambda d: d.get('affiliation') 
                                              if not pd.isna(d) else '' ).str.split(', ').apply(len)<=3,
                            ma['news_i_j'].apply(lambda d: d.get('WOS_author') 
                                              if not pd.isna(d) else '' ).str.split(' ').apply(len)<=2)]
#****

In [ ]:
ma['news_i_j_str']=ma['news_i_j'].astype(str)

In [ ]:
#Extract all the mathches for the author
kk=UDEAnewY.merge( ma[['news_i_j_str','news_i_j']],on='news_i_j_str',how='left')#.dropna().reset_index(drop=True)#.loc[5].values

In [ ]:
ma.UDEA_authors.str[i].loc[0]

In [ ]:
kk.shape,UDEAnewY.shape

In [ ]:
kkk=kk.dropna(subset=['news_i_j'])#.shape
kkk.shape